In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
!pip install wwo-hist

In [3]:
from wwo_hist import retrieve_hist_data

import os

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

In [5]:
#Raw content URL for adult-all.txt
url = "https://raw.githubusercontent.com/hastighsh/Ellehacks_2024/main/power_outages_data_copy.csv"

#Specify the delimiter (assuming it's a tab-separated file)
delimiter = ','

#Read the data into a DataFrame
power_outages_data = pd.read_csv(url, delimiter=delimiter)

#Let's create a backup copy of the dataset
outage_backup = power_outages_data.copy()

In [6]:
power_outages_data

,Unnamed: 0,Event Description,Year,Date Event Began,Date of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags,Time Event Began,Time of Restoration
0,0,Severe Weather - Thunderstorms,2014,6/30/2014,7/2/2014,Exelon Corporation/ComEd,Illinois,RFC,Unknown,"420,000","severe weather, thunderstorm",20:00:00,18:30:00
1,1,Severe Weather - Thunderstorms,2014,6/30/2014,7/1/2014,Northern Indiana Public Service Company,North Central Indiana,RFC,Unknown,"127,000","severe weather, thunderstorm",23:20:00,17:00:00
2,2,Severe Weather - Thunderstorms,2014,6/30/2014,7/1/2014,We Energies,Southeast Wisconsin,MRO,424,"120,000","severe weather, thunderstorm",17:55:00,02:53:00
3,3,Fuel Supply Emergency - Coal,2014,6/27/2014,Unknown,We Energies,Wisconsin,MRO,Unknown,Unknown,"fuel supply emergency, coal",13:21:00,NaN
4,4,Physical Attack - Vandalism,2014,6/24/2014,6/24/2014,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical",14:54:00,14:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647,1647,Transmission Line Loss,2000,3/18/2000,3/18/2000,El Paso Elec. Co.,Texas,MAIN,400,"100,000",transmission interruption,16:00:00,17:10:00
1648,1648,Vandalism,2000,3/14/2000,NaN,Alliant Energy,Maine,MAIN,NaN,NaN,vandalism,21:06:00,NaN
1649,1649,Ice Storm,2000,1/29/2000,2/3/2000,Duke Power Co.,South Carolina,SERC,300,"81,000","severe weather, winter storm",22:00:00,12:00:00
1650,1650,Ice Storm,2000,1/24/2000,NaN,Carolina Power & Light,North Carolina & Northern South Carolina,SERC,960,"173,000","severe weather, winter storm",19:00:00,NaN


In [7]:
power_outages_data = power_outages_data.drop(["Unnamed: 0", "NERC Region", "Demand Loss (MW)", "Number of Customers Affected"], axis=1)

In [8]:
# Count missing values in each column
missing_values = power_outages_data.isnull().sum()

print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Event Description       0
Year                    0
Date Event Began        0
Date of Restoration    14
Respondent              0
Geographic Areas        1
Tags                    1
Time Event Began       12
Time of Restoration    44
dtype: int64


In [9]:
cleaned_power_outages = power_outages_data.dropna()

In [10]:
cleaned_power_outages

,Event Description,Year,Date Event Began,Date of Restoration,Respondent,Geographic Areas,Tags,Time Event Began,Time of Restoration
0,Severe Weather - Thunderstorms,2014,6/30/2014,7/2/2014,Exelon Corporation/ComEd,Illinois,"severe weather, thunderstorm",20:00:00,18:30:00
1,Severe Weather - Thunderstorms,2014,6/30/2014,7/1/2014,Northern Indiana Public Service Company,North Central Indiana,"severe weather, thunderstorm",23:20:00,17:00:00
2,Severe Weather - Thunderstorms,2014,6/30/2014,7/1/2014,We Energies,Southeast Wisconsin,"severe weather, thunderstorm",17:55:00,02:53:00
4,Physical Attack - Vandalism,2014,6/24/2014,6/24/2014,Tennessee Valley Authority,"Nashville, Tennessee","vandalism, physical",14:54:00,14:55:00
5,Physical Attack - Vandalism,2014,6/19/2014,6/19/2014,Tennessee Valley Authority,"Nashville, Tennessee","vandalism, physical",08:47:00,08:48:00
...,...,...,...,...,...,...,...,...,...
1642,Severe Weather,2000,5/2/2000,5/2/2000,Reliant Energy HL&P,"Houston, TX",severe weather,04:00:00,12:00:00
1646,Transmission Line Loss,2000,3/18/2000,3/18/2000,Public Service of New Mexico,New Mexico,transmission interruption,19:08:00,19:08:00
1647,Transmission Line Loss,2000,3/18/2000,3/18/2000,El Paso Elec. Co.,Texas,transmission interruption,16:00:00,17:10:00
1649,Ice Storm,2000,1/29/2000,2/3/2000,Duke Power Co.,South Carolina,"severe weather, winter storm",22:00:00,12:00:00


In [11]:
# Count missing values in each column
missing_values = cleaned_power_outages.isnull().sum()

print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Event Description      0
Year                   0
Date Event Began       0
Date of Restoration    0
Respondent             0
Geographic Areas       0
Tags                   0
Time Event Began       0
Time of Restoration    0
dtype: int64


In [12]:
city_counts = cleaned_power_outages['Geographic Areas'].value_counts()

# Print the counts for each city
print(city_counts)

Island of Puerto Rico              30
California                         24
Texas                              20
Northern California                20
Tacoma, Washington                 17
                                   ..
Northeast, Illinois                 1
Jefferson, Oregon                   1
Fredrickson, Washington             1
Entire ComEd Territory, Indiana     1
New Mexico                          1
Name: Geographic Areas, Length: 985, dtype: int64


In [13]:
count_of_cities = len(city_counts)
print("Number of cities:", count_of_cities)

Number of cities: 985


In [14]:
top_20_cities = city_counts.head(20)
print("Top 20 cities:")
print(top_20_cities)

Top 20 cities:
Island of Puerto Rico                 30
California                            24
Texas                                 20
Northern California                   20
Tacoma, Washington                    17
Southeastern Michigan                 14
Illinois                              12
New York                              12
Southern California                   12
Northern and Central California       10
Georgia                               10
Pennsylvania                          10
Newark, Delaware                      10
San Francisco Bay Area, California    10
Northern Illinois                     10
Alberta, Canada                        9
Ohio                                   9
Virginia                               9
North Carolina                         8
Michigan                               8
Name: Geographic Areas, dtype: int64


In [179]:
# Assuming df is your DataFrame
# Split the "Event Description" column into two separate columns
cleaned_power_outages[['Event_Type_1', 'Event_Type_2']] = cleaned_power_outages['Event Description'].str.split(' - ', 1, expand=True)

# Drop the original "Event Description" column if needed
# df.drop('Event Description', axis=1, inplace=True)

# Print the first few rows to verify the changes
cleaned_power_outages

,Event Description,Year,Date Event Began,Date of Restoration,Respondent,Geographic Areas,Tags,Time Event Began,Time of Restoration,Event_Type_1,Event_Type_2
0,Severe Weather - Thunderstorms,2014,6/30/2014,7/2/2014,Exelon Corporation/ComEd,Illinois,"severe weather, thunderstorm",20:00:00,18:30:00,Severe Weather,Thunderstorms
1,Severe Weather - Thunderstorms,2014,6/30/2014,7/1/2014,Northern Indiana Public Service Company,North Central Indiana,"severe weather, thunderstorm",23:20:00,17:00:00,Severe Weather,Thunderstorms
2,Severe Weather - Thunderstorms,2014,6/30/2014,7/1/2014,We Energies,Southeast Wisconsin,"severe weather, thunderstorm",17:55:00,02:53:00,Severe Weather,Thunderstorms
4,Physical Attack - Vandalism,2014,6/24/2014,6/24/2014,Tennessee Valley Authority,"Nashville, Tennessee","vandalism, physical",14:54:00,14:55:00,Physical Attack,Vandalism
5,Physical Attack - Vandalism,2014,6/19/2014,6/19/2014,Tennessee Valley Authority,"Nashville, Tennessee","vandalism, physical",08:47:00,08:48:00,Physical Attack,Vandalism
...,...,...,...,...,...,...,...,...,...,...,...
1642,Severe Weather,2000,5/2/2000,5/2/2000,Reliant Energy HL&P,"Houston, TX",severe weather,04:00:00,12:00:00,Severe Weather,None
1646,Transmission Line Loss,2000,3/18/2000,3/18/2000,Public Service of New Mexico,New Mexico,transmission interruption,19:08:00,19:08:00,Transmission Line Loss,None
1647,Transmission Line Loss,2000,3/18/2000,3/18/2000,El Paso Elec. Co.,Texas,transmission interruption,16:00:00,17:10:00,Transmission Line Loss,None
1649,Ice Storm,2000,1/29/2000,2/3/2000,Duke Power Co.,South Carolina,"severe weather, winter storm",22:00:00,12:00:00,Ice Storm,None


In [16]:
# Get the top 20 cities
top_20_cities = city_counts.head(20).index

# Create a boolean mask indicating whether each row's city is in the top 20 cities
mask = cleaned_power_outages['Geographic Areas'].isin(top_20_cities)

# Filter the DataFrame to keep only the rows with cities in the top 20
df_top_20 = cleaned_power_outages[mask]

# Print the resulting DataFrame
print("DataFrame with rows for the top 20 cities only:")
df_top_20


DataFrame with rows for the top 20 cities only:


,Event Description,Year,Date Event Began,Date of Restoration,Respondent,Geographic Areas,Tags,Time Event Began,Time of Restoration,Event_Type_1,Event_Type_2
0,Severe Weather - Thunderstorms,2014,6/30/2014,7/2/2014,Exelon Corporation/ComEd,Illinois,"severe weather, thunderstorm",20:00:00,18:30:00,Severe Weather,Thunderstorms
13,Electrical System Islanding,2014,6/9/2014,6/9/2014,Peak Reliability,"Alberta, Canada",islanding,11:07:00,11:30:00,Electrical System Islanding,None
18,Electrical System Islanding,2014,6/3/2014,6/3/2014,Peak Reliability,"Alberta, Canada",islanding,15:32:00,15:59:00,Electrical System Islanding,None
19,Physical Attack - Vandalism,2014,6/3/2014,6/3/2014,Lower Colorado River Authority,Texas,"vandalism, physical",01:38:00,01:43:00,Physical Attack,Vandalism
22,Physical Attack - Vandalism,2014,5/23/2014,5/25/2014,Duke Energy Progress,North Carolina,"vandalism, physical",15:00:00,19:00:00,Physical Attack,Vandalism
...,...,...,...,...,...,...,...,...,...,...,...
1628,Severe Weather,2000,8/6/2000,8/7/2000,Commonwealth Edison,Illinois,severe weather,16:00:00,12:00:00,Severe Weather,None
1636,Severe Weather,2000,5/25/2000,6/2/2000,Duke Power,North Carolina,severe weather,10:00:00,06:00:00,Severe Weather,None
1637,Voltage Elec Usage,2000,5/24/2000,5/25/2000,Entergy,Texas,voltage reduction,10:15:00,22:14:00,Voltage Elec Usage,None
1640,Energy Conservation,2000,5/9/2000,5/9/2000,Consolidated Edison Co. of New York,New York,load shedding,11:39:00,23:00:00,Energy Conservation,None


In [17]:
# Select only the desired columns to create a smaller temporary DataFrame
temp_df = df_top_20[['Geographic Areas','Date Event Began', 'Date of Restoration']]

temp_df

,Geographic Areas,Date Event Began,Date of Restoration
0,Illinois,6/30/2014,7/2/2014
13,"Alberta, Canada",6/9/2014,6/9/2014
18,"Alberta, Canada",6/3/2014,6/3/2014
19,Texas,6/3/2014,6/3/2014
22,North Carolina,5/23/2014,5/25/2014
...,...,...,...
1628,Illinois,8/6/2000,8/7/2000
1636,North Carolina,5/25/2000,6/2/2000
1637,Texas,5/24/2000,5/25/2000
1640,New York,5/9/2000,5/9/2000


In [18]:
# Splitting the "Geographic Areas" column
# temp_df['Geographic Areas'] = temp_df['Geographic Areas'].str.split(',').str[0]
temp_df.loc[:, 'Geographic Areas'] = temp_df['Geographic Areas'].str.split(',').str[0]
temp_df

C:\Users\Heliiix\AppData\Local\Temp\ipykernel_6672\1480150531.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.loc[:, 'Geographic Areas'] = temp_df['Geographic Areas'].str.split(',').str[0]


,Geographic Areas,Date Event Began,Date of Restoration
0,Illinois,6/30/2014,7/2/2014
13,Alberta,6/9/2014,6/9/2014
18,Alberta,6/3/2014,6/3/2014
19,Texas,6/3/2014,6/3/2014
22,North Carolina,5/23/2014,5/25/2014
...,...,...,...
1628,Illinois,8/6/2000,8/7/2000
1636,North Carolina,5/25/2000,6/2/2000
1637,Texas,5/24/2000,5/25/2000
1640,New York,5/9/2000,5/9/2000


In [102]:

temp_df.reset_index(drop=True, inplace=True)

t = temp_df.head(len(temp_df))
t

,Geographic Areas,Date Event Began,Date of Restoration
0,Illinois,6/30/2014,7/2/2014
1,Alberta,6/9/2014,6/9/2014
2,Alberta,6/3/2014,6/3/2014
3,Texas,6/3/2014,6/3/2014
4,North Carolina,5/23/2014,5/25/2014
...,...,...,...
258,Illinois,8/6/2000,8/7/2000
259,North Carolina,5/25/2000,6/2/2000
260,Texas,5/24/2000,5/25/2000
261,New York,5/9/2000,5/9/2000


In [177]:
import urllib.parse
import os
import pandas as pd

# Assuming t is your DataFrame
for index, row in t.iterrows():
    print(index)
    # Extract values from the row
    api_key = 'd14fcf5b629c4c148f4191223241702'
    location = row['Geographic Areas']
    encoded_location = urllib.parse.quote(location)
    print(encoded_location)
    start_date = row['Date Event Began']
    end_date = row['Date of Restoration']  # Use a different date for the end date
    frequency = 24

    try:
        # Call retrieve_hist_data function to get historical weather data
        response = retrieve_hist_data(api_key=api_key,
                                       location_list=[encoded_location],
                                       start_date=start_date,
                                       end_date=end_date,
                                       frequency=frequency)
        
        if response is None:
            print(f"No data returned for location: {location}")
            continue  # Skip to the next iteration if no data is returned

        # Continue processing the data if it's successfully retrieved
        
        s = row['Geographic Areas'].replace(' ', '%20') + ".csv"
        df_excel = pd.read_csv(s)

        # Read existing data from combined_data.csv into another DataFrame
        combined_data_csv_path = 'combined_data.csv'
        if os.path.exists(combined_data_csv_path):
            df_combined_data = pd.read_csv(combined_data_csv_path)
        else:
            df_combined_data = pd.DataFrame()  # If combined_data.csv doesn't exist yet

        # Concatenate new data from Excel file with existing data from combined_data.csv
        combined_df = pd.concat([df_combined_data, df_excel], ignore_index=True)

        # Write the combined DataFrame back to combined_data.csv
        combined_df.to_csv(combined_data_csv_path, index=False)
        
    except Exception as e:
        print(f"Error processing data for location: {location}. Error: {e}")
        continue  # Skip to the next iteration if an error occurs


0
Illinois


Retrieving weather data for Illinois


Currently retrieving data for Illinois: from 2014-06-30 to 2014-06-30


C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.436682
Currently retrieving data for Illinois: from 2014-07-01 to 2014-07-02
Time elapsed (hh:mm:ss.ms) 0:00:00.708249


export Illinois completed!


1
Alberta


Retrieving weather data for Alberta


Currently retrieving data for Alberta: from 2014-06-09 to 2014-06-09


C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.273881


export Alberta completed!


2
Alberta


Retrieving weather data for Alberta


Currently retrieving data for Alberta: from 2014-06-03 to 2014-06-03


...

Error processing data for location: Texas. Error: 'weather'
261
New%20York


Retrieving weather data for New%20York


Currently retrieving data for New%20York: from 2000-05-09 to 2000-05-09


C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Error processing data for location: New York. Error: 'weather'
262
Texas


Retrieving weather data for Texas


Currently retrieving data for Texas: from 2000-03-18 to 2000-03-18


C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Error processing data for location: Texas. Error: 'weather'
